In [ ]:
!pip install "arize-phoenix[evals]" "langchain>=0.0.334" "openai>=1" chromadb tiktoken playwright asyncio nest_asyncio

In [ ]:
import os
from getpass import getpass

import openai
from langchain.indexes import VectorstoreIndexCreator

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
import asyncio
import os
from getpass import getpass

import nest_asyncio
import openai
import phoenix as px
from langchain.chat_models import ChatOpenAI
from phoenix.trace.langchain import LangChainInstrumentor

In [ ]:
LangChainInstrumentor().instrument()

In [ ]:
px.launch_app()

In [ ]:
nest_asyncio.apply()

In [ ]:
!playwright install

In [ ]:
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import BeautifulSoupTransformer

# Load HTML
loader = AsyncChromiumLoader(["https://www.bogleheads.org/wiki/Asset_protection"])

In [ ]:
asyncio.set_event_loop_policy(asyncio.DefaultEventLoopPolicy())

In [ ]:
html = loader.load()

In [ ]:
# Transform
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["p", "li", "div", "a"])

In [ ]:
index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
from phoenix.evals import OpenAIModel

questions = []
model = OpenAIModel("gpt-3.5-turbo", temperature=2)
random_question_prompt = """
"Generate a random financial question. Don't repeat a question. it can be about
asset protection, the stock market or other
"""
for i in range(5):
    question = model.generate(prompts=[])[0]
    print(question)
    questions.append(question)

In [ ]:
questions

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-4")

In [ ]:
for question in questions:
    answer = index.query_with_sources(question, llm=llm)
    print(answer)